In [7]:
import os
import glob
import zlib
import pathlib
import tempfile
import subprocess
import numpy as np

from binascii import unhexlify
from xml.etree import ElementTree
from struct import unpack


In [8]:

# island_glob = os.path.join("..", "resources", "data", "base", "provinces", "roman", "islands", "pool", "**", "*.a7m")
island_glob = os.path.join("..", "resources", "1800", "data", "sessions", "islands", "pool", "moderate", "**", "*.a7m")
# island_glob = os.path.join("..", "resources", "1800", "data", "sessions", "islands", "pool", "moderate", "moderate_l_01", "*.a7m")
output_dir = os.path.join("..", "resources", "islands")
reader_path = os.path.join("..", "lib", "FileDBReader", "FileDBReader.exe")

In [9]:
def enumerate_bits(byte):
    for i in range(8):
        yield byte >> i & 1


import matplotlib.pyplot as plt

def read_grids(xmlpath):
    data = ElementTree.parse(xmlpath)
    node = data.getroot()

    # Estimate the islands with river by calculating how many river tiles block land tiles.
    # The result will slightly underestimate land area because we subtract coast river tiles.

    res = node.find(".//Water")
    width = unpack("I", unhexlify(res.find("x").text))[0]
    height = unpack("I", unhexlify(res.find("y").text))[0]
    water = unhexlify(res.find("bits").text)
    water = np.array([bit for byte in water for bit in enumerate_bits(byte)], dtype=float)
    water = water.reshape(width, height)
    water[water == 0] = np.nan

    res = node.find(".//AreaIDs")
    width = unpack("I", unhexlify(res.find("x").text))[0]
    height = unpack("I", unhexlify(res.find("y").text))[0]
    area = unhexlify(res.find("val").text)
    area = np.array(unpack(f"{len(area) // 2}h", area), dtype=float)
    area = area.reshape(width, height)
    area[area != 8193] = np.nan  # Keep only the buildable area.
    area[area == 8193] = 1

    res = node.find(".//RiverGrid")
    width = unpack("I", unhexlify(res.find("x").text))[0]
    height = unpack("I", unhexlify(res.find("y").text))[0]
    river = unhexlify(res.find("bits").text)
    river = np.array([bit for byte in river for bit in enumerate_bits(byte)], dtype=float)
    river = river.reshape(width, height)
    river[river != 0] = np.nan
    river[river == 0] = 1

    return area, water, river

In [10]:
def a7m_to_xml(data_compressed):
    decompressed = zlib.decompress(data_compressed)
    with tempfile.TemporaryDirectory() as tmp:
        decompressed_path = os.path.join(tmp, "data.a7m")
        with open(decompressed_path, "wb") as f:
            f.write(decompressed)
        read_cmd = [reader_path, "decompress", "-f", decompressed_path]
        read_process = subprocess.Popen(read_cmd)
        read_return_code = read_process.wait()
        if read_return_code != 0:
            raise Exception(read_return_code)
        xml_path = os.path.join(tmp, "data.xml")
        area, water, river = read_grids(xml_path)
    return area, water, river

In [11]:
def read_island(island_path):
	with open(island_path, "rb") as f:
		f.seek(0x310)
		offset = unpack("Q", f.read(8))[0]
		size = offset - 0x318
		payload = f.read(size)
	return payload

In [ ]:
for input_path in glob.glob(island_glob):
    island_name = pathlib.Path(input_path).stem
    output_path = os.path.join(output_dir, island_name + ".npy")
    
    print(f"Converting {input_path} \u2192 {output_path}", end="")
    island_data = read_island(input_path)
    area, water, river = a7m_to_xml(island_data)

    buildable = (area == 1) & (water == 1)
    np.save(output_path, buildable)
    print(" \u2713")

Converting ..\resources\1800\data\sessions\islands\pool\moderate\community_island\community_island.a7m → ..\resources\islands\community_island.npy ✓
Converting ..\resources\1800\data\sessions\islands\pool\moderate\community_island\community_island_river_01.a7m → ..\resources\islands\community_island_river_01.npy ✓
Converting ..\resources\1800\data\sessions\islands\pool\moderate\moderate_3rdparty02_01\moderate_3rdparty02_01.a7m → ..\resources\islands\moderate_3rdparty02_01.npy ✓
Converting ..\resources\1800\data\sessions\islands\pool\moderate\moderate_3rdparty03_01\moderate_3rdparty03_01.a7m → ..\resources\islands\moderate_3rdparty03_01.npy ✓
Converting ..\resources\1800\data\sessions\islands\pool\moderate\moderate_3rdparty07_01\moderate_3rdparty07_01.a7m → ..\resources\islands\moderate_3rdparty07_01.npy ✓
Converting ..\resources\1800\data\sessions\islands\pool\moderate\moderate_3rdparty08_01\moderate_3rdparty08_01.a7m → ..\resources\islands\moderate_3rdparty08_01.npy ✓
Converting ..\re